## Phase 1 Time plot

time plot for phase 1

In [1]:
#imports
import math
import numpy as np
import pandas as pd
import os
import sys
import subprocess
import matplotlib as plt
import seaborn as sns
%matplotlib qt

In [2]:
xlsx=True#No text files exist for phase 1, so this should remain True

In [3]:
#load in files
files_raw = str(subprocess.check_output(['ls', 'Phase1/']))
files = files_raw.split('\\n')

files[0] = files[0][2:]#remove "b' from beginning
files=files[:-1]#remove non-entry last item

filelist=[]
#get rid of xlsx files from file selection
if xlsx==False:
    for filename in files:
        if filename[-4:] == '.txt':
            filelist.append(filename)
#get rid of txt files from file selection
if xlsx==True:
    for filename in files:
        if filename[-4:] == 'xlsx':
            filelist.append(filename)
            
            
#manipulate names in files, create observations list
observations = []
for i in range(0,len(filelist)):
    if xlsx==False:
        observations.append(filelist[i][8:-15])
    if xlsx==True:
        observations.append(filelist[i][:-5])

#remove bad date = only the 56655 one because I don't know which 56655 observation is the good one
#as for 56662: theoretical max not computing correctly. data is larger than
observations.remove('sb27260261.eb28597220.56655.44330324074.tar')
#observations.remove('sb27262884.eb28597392.56656.24319659722.tar')
observations.remove('sb27264735.eb28612294.56662.21643457176.tar')
        
obs_dfs = {}#dictionary of dataframes

#read in files
for obs in observations:
    if obs not in obs_dfs.keys():
        if xlsx==False:
            obs_dfs[obs] = pd.read_table('Phase1/13B-266.'+obs+'.flag_stats.txt',sep=' ')
            #fix each dataframe
            obs_dfs[obs].drop(['#total','%flagged'],axis=1,inplace=True)
            obs_dfs[obs].columns = ['freq','#flags','#total','%flagged']
            obs_dfs[obs].set_index('freq',inplace=True)
        if xlsx==True:
            obs_dfs[obs] = pd.read_excel('Phase1/'+obs+'.xlsx')
            #obs_dfs[obs].set_index('freq',inplace=True)
            obs_dfs[obs] = obs_dfs[obs].drop('freq')

            
#clear some variables
files_raw=None
files=None
filelist=None

#fix the observation names (keys for dictionary obs_dfs) to be the MJD. Will be used for the time in the final plot.
dates=[]
#sort by MJD (found in the middle of the filename)
observations.sort(key=lambda x:x[obs.find('.',obs.find('.')+1)+1:obs.find('.',obs.find('.')+1)+6])



for i in range(0,len(observations)):
    obs=observations[i]
    strt = obs.find('.',obs.find('.')+1)
    dates.append(obs[strt+1:strt+6])
dates=np.sort(dates)



In [4]:
#load in epoch 1 archive data
archive_df = pd.read_excel('archive_epoch1_edited.xlsx')

#turn into array for doing np methods
archive_arr = archive_df.as_matrix(['TOS','Tele:config','First Time'])

#The edited archive only has the observations that werent taken out in cell 3

#add up total amount of theoretical visibilities
theor_max_list = []
for i in range(0,len(archive_arr), 19):
    #baselines and polarization (N)(N-1)*2/2
    blines = int(archive_arr[i,1][-2:])*(int(archive_arr[i,1][-2:])-1)
    theor_max_list.append((blines*(archive_arr[i,0]/8)))

In [5]:
#on good faith that both theor_max_list and the session dataframes are sorted by date and match up:
#normalize each one by the theoretical max:
norm_dfs={}

for i in range(0,40):
    norm_flip = lambda x: 1 - (x/theor_max_list[i]) #divide by theoretical max, 1 minus to fit phase 2 data
    norm_dfs[observations[i]] = obs_dfs[observations[i]].apply(norm_flip,axis=1)
    #if i == 28:
        #wtf = obs_dfs[observations[i]].apply(norm_flip,axis=1)
    #data_list.append(list(norm_df['npts_tot']))

#big_df=pd.DataFrame(data_list,index=dates,columns=list(obs_dfs[obs].index.values))


In [6]:
#put together big df
big_df=pd.concat(norm_dfs,axis=1)
big_df=big_df.transpose()
big_df.index=dates
big_df
#frequencies: 944.0 -> 1419.984375 in 30464 channels

,944.0,944.015625,944.03125,944.046875,944.0625,944.078125,944.09375,944.109375,944.125,944.140625,...,1419.84375,1419.859375,1419.875,1419.890625,1419.90625,1419.921875,1419.9375,1419.953125,1419.96875,1419.984375
56590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.226963,0.227084,0.227386,0.227545,0.228891,0.230865,0.232920,0.233897,0.233881,0.233562
56596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.160679,0.160838,0.160980,0.161026,0.160708,0.160309,0.160406,0.160672,0.160844,0.160834
56605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.186074,0.185903,0.185482,0.185306,0.185309,0.185577,0.186026,0.186230,0.186187,0.186168
56608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.215619,0.215244,0.214732,0.214691,0.214686,0.214979,0.215587,0.216039,0.215962,0.215235
56612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199960,0.199887,0.199754,0.199521,0.199416,0.199681,0.200097,0.200280,0.199997,0.199795
56613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.309081,0.308976,0.309008,0.309044,0.309090,0.309218,0.309264,0.309264,0.309296,0.309355
56614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.267348,0.267791,0.267988,0.267819,0.267522,0.267289,0.267412,0.267618,0.267773,0.267677
56615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.204110,0.204188,0.204298,0.204225,0.204156,0.204165,0.204115,0.204023,0.203987,0.204028
56617,0.718705,0.719838,0.730013,0.760863,0.808291,0.847267,0.864700,0.863361,0.845864,0.814955,...,0.224742,0.225350,0.226191,0.227042,0.228038,0.229409,0.230552,0.231768,0.232824,0.234232
56618,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.193697,0.193912,0.194273,0.194666,0.195173,0.196096,0.197289,0.198089,0.198268,0.198450


In [7]:
#change NaNs to 0
big_arr=np.array(big_df)
for i in range(0,40):
    for j in range(0,30464):
        if math.isnan(big_arr[i,j]):
            big_arr[i,j]=1.0
fixed_df=pd.DataFrame(big_arr)
fixed_df.columns=big_df.columns

In [8]:
#make smaller for sns
#very bad way of averaging - there has to be an automatic decimator somewhere
trunc_list=[]

for mjd in fixed_df.index.values:
    ser = fixed_df.loc[mjd,:]
    #trunc_list.append(ser.rolling(window=5).mean())#this doesn't have a decimate option so Im doing it manually
    avgs_2 = [(a + b) / 2 for a, b in zip(*[iter(list(ser))]*2)]
    avgs_4 = [(a + b) / 2 for a, b in zip(*[iter(list(avgs_2))]*2)]
    avgs_8 = [(a + b) / 2 for a, b in zip(*[iter(list(avgs_4))]*2)]
    avgs_16 = [(a + b) / 2 for a, b in zip(*[iter(list(avgs_8))]*2)]
    trunc_list.append(avgs_16)

    
trunc_cols=list(fixed_df.columns.values)[::16]
trunc_df=pd.DataFrame(trunc_list,index=dates,columns=trunc_cols)

In [10]:
sns.heatmap(trunc_df,xticklabels=True)

In [9]:
#save dataframes if desired
#writer=pd.ExcelWriter('phase1_waterfall_full.xlsx',engine='xlsxwriter')
#fixed_df.to_excel(writer, sheet_name='Sheet1')
#writer.save()

fixed_df.to_csv('phase1_waterfall_full.csv')

writer=pd.ExcelWriter('phase1_waterfall_trun.xlsx',engine='xlsxwriter')
trunc_df.to_excel(writer, sheet_name='Sheet1')
writer.save()

-----